In [ ]:
"""
긍정 기사 상위 104857개, 부정 기사 상위 104857개, 중립 기사 상위 104857개를 각각 하나의 파일로 생성한다. 
104857개는 전체 기사 10784446개의 1%이다. 
위 파일은 ELECTRA sentiment model fine-tuning을 위한 파일이다. 
상위 기사를 따로 추출하는 이유는, 확실히 긍정, 부정, 중립인 기사를 모델 학습에 사용하기 위해서이다. 
"""


import glob
import os
import pandas as pd

top = 104857
qlist = ["여성", "여자+-여성", "남성", '남자+-남성']

df_pos = pd.DataFrame(columns=['title', 'source', 'date', 'text', "knu_score"])
df_neg = pd.DataFrame(columns=['title', 'source', 'date', 'text', "knu_score"])
df_neu = pd.DataFrame(columns=['title', 'source', 'date', 'text', "knu_score"])


df_pos_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_knu_score', f"{'knu_score_pos'}.xlsx")
df_neg_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_"knu_score', f"{'knu_score_neg'}.xlsx")
df_neu_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_"knu_score', f"{'knu_score_neu'}.xlsx")


df_list = []
for query in qlist:
    print(query)
    path = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_knu_score', '#3_'+query)
    g = map(pd.read_excel, glob.glob(path+"/*.xlsx"))
    d = pd.concat(list(g), ignore_index=True)
    d = d.drop(columns=['url'])
    df_list.append(d)

print('concat')
df = pd.concat(df_list, ignore_index=True)
del df_list
print('df_pos')
df_pos = df.nlargest(top, "knu_score")
print('df_neg')
df_neg = df.nsmallest(top, "knu_score")
print('df_neu')
df_neu = df[df['knu_score'] == 0].sample(n=top)


print('to_excel')
writer_pos = pd.ExcelWriter(df_pos_name, engine='xlsxwriter')
df_pos.to_excel(writer_pos)
writer_pos.book.use_zip64()
writer_pos.save()

writer_neg = pd.ExcelWriter(df_neg_name, engine='xlsxwriter')
df_neg.to_excel(writer_neg)
writer_neg.book.use_zip64()
writer_neg.save()

writer_neu = pd.ExcelWriter(df_neu_name, engine='xlsxwriter')
df_neu.to_excel(writer_neu)
writer_neu.book.use_zip64()
writer_neu.save()